# Build the Neural Network

In the following, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

1. Get device for Training :

We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available. Let’s check to see if `torch.cuda` or `torch.backends.mps` are available, otherwise we use the CPU.

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


2. Define the class :

- We define our neural network by subclassing `nn.Module`.
- We initialize the neural network layers in `__init__`.
- We implements the operations on input data in the `forward` method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [22]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3])


In [23]:
pred_probab

tensor([[0.0968, 0.0965, 0.0923, 0.1092, 0.1039, 0.0993, 0.0963, 0.1063, 0.1052,
         0.0942]], grad_fn=<SoftmaxBackward0>)

3. Model Layers :

In [20]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten :

In [32]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear :

In [33]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU :

In [34]:
print(f"Before ReLU: {hidden1}\n\n")

hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1309, -0.1936, -0.2235,  0.1022,  0.6131,  0.4898,  0.2966, -0.1828,
         -0.5248,  0.1203,  0.1041,  0.2717, -0.3834,  0.1556,  0.6499, -0.3059,
          0.0618,  0.0554, -0.4135,  0.0988],
        [ 0.0886, -0.3835, -0.1548,  0.1037,  0.4082,  0.5556,  0.2976, -0.3366,
         -0.3224,  0.1293,  0.0684,  0.2142, -0.7134,  0.1357,  0.4091, -0.5352,
         -0.0928, -0.2439, -0.2654, -0.0201],
        [ 0.5758, -0.1354, -0.1735,  0.1753,  0.1884,  0.0143,  0.4698, -0.3478,
         -0.2622,  0.2922,  0.1715,  0.1591, -0.4034,  0.0622,  0.7671, -1.0089,
          0.1153, -0.0082, -0.3142,  0.1288]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.1022, 0.6131, 0.4898, 0.2966, 0.0000, 0.0000,
         0.1203, 0.1041, 0.2717, 0.0000, 0.1556, 0.6499, 0.0000, 0.0618, 0.0554,
         0.0000, 0.0988],
        [0.0886, 0.0000, 0.0000, 0.1037, 0.4082, 0.5556, 0.2976, 0.0000, 0.0000,
         0.1293, 0.0684, 0.2142, 0.0000, 0.1357, 0.40

nn.Sequential :

In [35]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [36]:
logits.size()

torch.Size([3, 10])

nn.Softmax :

logits contains raw values in [-infty, infty] - which are passed to the `nn.Softmax` module to be  scaled to values [0, 1] representing the model’s predicted probabilities for each class.

`dim` parameter indicates the dimension along which the values must sum to 1.

In [41]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

4. Model Parameters :    

They're accessible using model’s `parameters()` or `named_parameters()` methods.

In [44]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0076, -0.0220, -0.0017,  ...,  0.0345,  0.0302,  0.0338],
        [-0.0012, -0.0291,  0.0081,  ...,  0.0213,  0.0354, -0.0324]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0173, 0.0196], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0154, -0.0219, -0.0059,  ..., -0.0420, -0.0391,  0.0113],
        [-0.0407,  0.0394, -0.0367,  ..., -0.0206,  0.0062, -0.0125]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si